**Import required libraries and scripts**

In [ ]:
#Import required libraries and scripts
from scripts.library_preparation import *
from scripts.utilities import *
from scripts.docking_functions import *
from scripts.clustering_functions import *
from scripts.rescoring_functions import *
from scripts.ranking_functions import *
import numpy as np
import os

**Define initial variables**
- software : path to the software directory cloned from the git repository
- protein_file : path to the file containing the prepared protein/receptor (.pdb format)
- ref_file : path to the file containing the reference ligand used for binding pocket definition (.mol2 format)
- docking_library : path to the file containing the compounds to be used for docking (.sdf format)
- id_column : name of the column containing a unique identifier for each compound in the docking_library file (string)
- n_poses : number of to generate for each compound
- exhaustiveness : exhaustiveness of the docking algorithm (GNINA and SMINA only)

The software will then create a temporary directory to store the output of the various functions.

In [ ]:
software = '/home/tony/CADD22/software'
protein_file = '/home/tony/CADD22/wocondock_refactored/2o1x_A_apo_protoss.pdb'
ref_file = '/home/tony/CADD22/wocondock_refactored/2o1x_A_lig_protoss.sdf'
docking_library = '/home/tony/CADD22/wocondock_refactored/Selection_of_FCHGroup_LeadLike.sdf'
id_column = 'ID'
n_poses = 10
exhaustiveness = 8

#Initialise variables and create a temporary folder
w_dir = os.path.dirname(protein_file)
print('The working directory has been set to:', w_dir)
create_temp_folder(w_dir+'/temp')

In [ ]:
from scripts.get_pocket import *
test = GetPocket(ref_file, protein_file, 8)

from scripts.dogsitescorer import *
pocket_coordinates2=binding_site_coordinates_dogsitescorer(protein_file, w_dir, method='volume')
pocket_coordinates2

**Library preparation**

This function will first standardize the library using the ChemBL structure pipeline. This will remove salts and make the library consistent.
Then a single protonation state is predicted using pka_solver. This step may take a while depending on the number of compounds.
Finally, one conformer is generated per molecule using GypsumDL.
The final_library is then written to a file in the main directory (final_library.sdf)

In [ ]:
cleaned_df = prepare_library_GypsumDL(docking_library, id_column, software)

In [ ]:
display(cleaned_df)

**Docking**

This function will dock all compounds in the receptor, using the reference ligand as a way to define the binding site. The docking results are written to the temporary folder. 

In [ ]:
all_poses = docking(protein_file, ref_file, id_column, software, exhaustiveness, n_poses)

In [ ]:
all_poses

**Clustering**

We will first load all the poses generated from the docking run. The cluster() function performs the calculation of the clustering metrics (for now simpleRMSD and electroshape similarity), then performs the clustering using the k-medoids clustering algorithm with the number of clusters optimised using silhouette score. Finally, all cluster centers are collected and written to a file in the temporary directory (/temp/clustering/) (one file per clustering metric).

In [ ]:
cluster(['RMSD'], w_dir, id_column, protein_file)
cluster(['espsim'], w_dir, id_column, protein_file)
cluster(['spyRMSD'], w_dir, id_column, protein_file)
cluster(['USRCAT'], w_dir, id_column, protein_file)

**Rescoring**

The file containing all the cluster centers is then rescored using all scoring functions available (GNINA, Vina, AutoDock4, PLP, CHEMPLP, RF-Score-VS). The rescored output is return as a dataframe.

In [ ]:
RMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/RMSD_kS_full.sdf')
espsim_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/espsim_d_kS_full.sdf')
spyRMSD_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/spyRMSD_kS_full.sdf')
USRCAT_rescored = rescore_all(w_dir, protein_file, ref_file, software, w_dir+'/temp/clustering/usr_kS_full.sdf')

**Final ranking methods**

This code calculates the final ranking of compounds using various methods.
*Method 1* : Calculates ECR value for each cluster center, then outputs the top ranked center.
*Method 2* : Calculates ECR value for each cluster center, then outputs the average ECR value for each compound.
*Method 3* : Calculates the average rank of each compound, then ouputs the corresponding ECR value for each compound.
*Method 6* : Calculates Z-score for each cluster center, then ouputs the top ranked center.
*Method 7* : Calculates Z-score for each cluster center, then ouputs the average Z-score for each compound.

All methods are then combined into a single dataframe for comparison purposes.

In [ ]:
apply_ranking_methods(w_dir)